## Clonación de voz usando Coqui

https://www.youtube.com/watch?v=sMp8a7WPQ0Q&ab_channel=FRIKIdelTO

In [5]:
import torch
import importlib

# Lista de clases que necesitan ser añadidas a los globales seguros
classes_to_make_safe = [
    "TTS.tts.configs.xtts_config.XttsConfig",      # La que ya estaba
    "TTS.tts.models.xtts.XttsAudioConfig",         # La que causó el error
    "TTS.config.shared_configs.BaseDatasetConfig", # Comúnmente necesaria para XTTS
    "TTS.tts.models.xtts.XttsArgs"                 # Comúnmente necesaria para XTTS
]

added_classes_details = []
successful_adds = 0

print(f"Configurando el entorno para cargar el modelo XTTS v2...")
for class_path_str in classes_to_make_safe:
    print(f"Intentando preparar para añadir: {class_path_str}")
    try:
        module_path, class_name = class_path_str.rsplit('.', 1)
        
        # Intentar importar el módulo y obtener la clase
        module = importlib.import_module(module_path)
        actual_class = getattr(module, class_name)
        
        # Añadir a los globales seguros
        # torch.serialization.add_safe_globals es idempotente, no hay problema si se llama varias veces con la misma clase
        torch.serialization.add_safe_globals([actual_class])
        
        added_classes_details.append(f"'{class_path_str}' procesada para globales seguros.")
        successful_adds +=1 # Contamos como éxito incluso si ya estaba, pues el estado deseado se alcanza
    except ImportError:
        added_classes_details.append(f"ERROR Importando: No se pudo importar el módulo para '{class_path_str}'. Verifica el path y la instalación de TTS.")
    except AttributeError:
        added_classes_details.append(f"ERROR Atributo: No se pudo encontrar la clase '{class_name}' en '{module_path}'.")
    except Exception as e:
        added_classes_details.append(f"ERROR Inesperado al procesar '{class_path_str}': {e}")

# Imprimir resumen
print("\nResumen del proceso de configuración de globales seguros:")
for detail in added_classes_details:
    print(f"INFO: {detail}")

print(f"\nINFO: {successful_adds} de {len(classes_to_make_safe)} clases fueron procesadas para los globales seguros de PyTorch.")
if successful_adds < len(classes_to_make_safe):
    print("ADVERTENCIA: No todas las clases pudieron ser procesadas. Revisa los mensajes de ERROR anteriores.")


Configurando el entorno para cargar el modelo XTTS v2...
Intentando preparar para añadir: TTS.tts.configs.xtts_config.XttsConfig
Intentando preparar para añadir: TTS.tts.models.xtts.XttsAudioConfig
Intentando preparar para añadir: TTS.config.shared_configs.BaseDatasetConfig
Intentando preparar para añadir: TTS.tts.models.xtts.XttsArgs

Resumen del proceso de configuración de globales seguros:
INFO: 'TTS.tts.configs.xtts_config.XttsConfig' procesada para globales seguros.
INFO: 'TTS.tts.models.xtts.XttsAudioConfig' procesada para globales seguros.
INFO: 'TTS.config.shared_configs.BaseDatasetConfig' procesada para globales seguros.
INFO: 'TTS.tts.models.xtts.XttsArgs' procesada para globales seguros.

INFO: 4 de 4 clases fueron procesadas para los globales seguros de PyTorch.


In [6]:
from TTS.api import TTS
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name_v2 = "tts_models/multilingual/multi-dataset/xtts_v2" # o la que estés usando para v2

print(f"\nIntentando cargar el modelo: {model_name_v2}...")
try:
    tts_v2 = TTS(model_name=model_name_v2).to(device) # Asegúrate que la variable sea tts_v2 o la que uses
    print("¡Modelo XTTS v2 cargado exitosamente!")
except Exception as e:
    print(f"ERROR al cargar el modelo '{model_name_v2}': {type(e).__name__} - {e}")
    print("Si es un UnpicklingError, revisa el mensaje para ver si falta otra clase en 'classes_to_make_safe' en la celda anterior.")
    print("Si es otro error, por favor copia el mensaje completo.")


Intentando cargar el modelo: tts_models/multilingual/multi-dataset/xtts_v2...
 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Using model: xtts
¡Modelo XTTS v2 cargado exitosamente!
¡Modelo XTTS v2 cargado exitosamente!


In [36]:
telematica = "Ingeniería Civil Telemática es una carrera que busca integrar las tecnologías, enfocándose en el desarrollo de soluciones innovadoras"

In [ ]:
tts_v2.tts_to_file(
    text=telematica,
    speaker_wav="audio_in/iron_voice.WAV",  # Puedes especificar un archivo de voz si es necesario
    language="es",  # Puedes especificar un idioma si es necesario
    file_path="TONY_telematica.wav"
)

 > Text splitted to sentences.
['Ingeniería Civil Telemática es una carrera que busca integrar las tecnologías, enfocándose en el desarrollo de soluciones innovadoras']
 > Processing time: 24.98600959777832
 > Real-time factor: 2.0692172632016255
 > Processing time: 24.98600959777832
 > Real-time factor: 2.0692172632016255


'TONY_telematica_2.wav'

In [39]:
quien = "Soy un prototipo de asistente virtual, pero me encantaría evolucionar para poder asistir junto a ustedes a ferias vocacionales o eventos como puertas abiertas aquí en la universidad, siendo parte de los increíbles proyectos creado por ustedes, el equipo de Difusión Telemática."

In [ ]:
tts_v2.tts_to_file(
    text=quien,
    speaker_wav="audio_in/iron_voice.WAV",  # Puedes especificar un archivo de voz si es necesario
    language="es",  # Puedes especificar un idioma si es necesario
    file_path="TONY_soy.wav"
)

 > Text splitted to sentences.
['Soy un prototipo de asistente virtual, pero me encantaría evolucionar para poder asistir junto a ustedes a ferias vocacionales o eventos como puertas abiertas aquí en la universidad, siendo parte de los increíbles proyectos creado por ustedes, el equipo de Difusión Telemática.']
[!] Warning: The text length exceeds the character limit of 239 for language 'es', this might cause truncated audio.
[!] Warning: The text length exceeds the character limit of 239 for language 'es', this might cause truncated audio.
 > Processing time: 42.114299058914185
 > Real-time factor: 2.0834536522441796
 > Processing time: 42.114299058914185
 > Real-time factor: 2.0834536522441796


'TONY_soy.wav'

In [66]:
familiar = "Esta voz les suena familiar porque el audio base para entrenar el modelo TTS, viene de un personaje ficticio que fue un genio, millonario, playboy y filántropo ¿Adivinaron cuál es el personaje?"

In [ ]:
tts_v2.tts_to_file(
    text=familiar,
    speaker_wav="audio_in/iron_voice.WAV",  # Puedes especificar un archivo de voz si es necesario
    language="es",  # Puedes especificar un idioma si es necesario
    file_path="TONY_soy_iron_man.wav"
)

 > Text splitted to sentences.
['Esta voz les suena familiar porque el audio base para entrenar el modelo TTS, viene de un personaje ficticio que fue un genio, millonario, playboy y filántropo ¿Adivinaron cuál es el personaje?']
 > Processing time: 33.31199049949646
 > Real-time factor: 1.9980017803507226
 > Processing time: 33.31199049949646
 > Real-time factor: 1.9980017803507226


'TONY_soy_iron_man.wav'

In [54]:
curioso = "Es curioso, porque antes programaba asistentes virtuales como Jarvis o Friday y ahora soy uno de ellos"

In [ ]:
tts_v2.tts_to_file(
    text=curioso,
    speaker_wav="audio_in/iron_voice.WAV",  # Puedes especificar un archivo de voz si es necesario
    language="es",  # Puedes especificar un idioma si es necesario
    file_path="TONY_curioso.wav"
)

 > Text splitted to sentences.
['Es curioso, porque antes programaba asistentes virtuales como Jarvis o Friday y ahora soy uno de ellos']
 > Processing time: 17.106234550476074
 > Real-time factor: 1.9359882967787503
 > Processing time: 17.106234550476074
 > Real-time factor: 1.9359882967787503


'TONY_curiso.wav'

In [56]:
saludo = "Hola chicos, soy Tony Stark, pero me pueden llamar TONY. Estoy aquí para ayudarles a conocer más sobre la carrera de Ingeniería Civil Telemática."

In [ ]:
tts_v2.tts_to_file(
    text=saludo,
    speaker_wav="audio_in/iron_voice.WAV",  # Puedes especificar un archivo de voz si es necesario
    language="es",  # Puedes especificar un idioma si es necesario
    file_path="TONY_saludando.wav"
)

 > Text splitted to sentences.
['Hola chicos, soy Tony Stark, pero me pueden llamar TONY.', 'Estoy aquí para ayudarles a conocer más sobre la carrera de Ingeniería Civil Telemática.']
 > Processing time: 19.52146005630493
 > Real-time factor: 1.849512727904251
 > Processing time: 19.52146005630493
 > Real-time factor: 1.849512727904251


'TONY_saludando.wav'

In [ ]:
cierre = "Espero que disfruten de la experiencia y aprendan mucho sobre nuestra carrera. ¡Nos vemos pronto!"

In [ ]:
tts_v2.tts_to_file(
    text=cierre,
    speaker_wav="audio_in/iron_voice.WAV",  # Puedes especificar un archivo de voz si es necesario
    language="es",  # Puedes especificar un idioma si es necesario
    file_path="TONY_cierre.wav"
)

 > Text splitted to sentences.
['Espero que disfruten de la experiencia y aprendan mucho sobre nuestra carrera.', '¡Nos vemos pronto!']
 > Processing time: 12.731318235397339
 > Real-time factor: 1.9577491567905554
 > Processing time: 12.731318235397339
 > Real-time factor: 1.9577491567905554


'TONY_cierre.wav'

In [69]:
cierre_presentacion = "Espero que hayan disfrutado de la presentación, los queremos tres millones"

In [72]:
tts_v2.tts_to_file(
    text=cierre_presentacion,
    speaker_wav="audio_in/iron_voice.WAV",  # Puedes especificar un archivo de voz si es necesario
    language="es",  # Puedes especificar un idioma si es necesario
    file_path="audio_out/TONY_cierre_presentacion_2.wav"
)

 > Text splitted to sentences.
['Espero que hayan disfrutado de la presentación, los queremos tres millones']
 > Processing time: 10.048422574996948
 > Real-time factor: 1.8610378122789504
 > Processing time: 10.048422574996948
 > Real-time factor: 1.8610378122789504


'audio_out/TONY_cierre_presentacion_2.wav'

In [79]:
apagar = "Hasta pronto! Cerrando sesión"

In [80]:
tts_v2.tts_to_file(
    text=apagar,
    speaker_wav="audio_in/iron_voice.WAV",  # Puedes especificar un archivo de voz si es necesario
    language="es",  # Puedes especificar un idioma si es necesario
    file_path="audio_out/cerrando_programa.wav"
)

 > Text splitted to sentences.
['Hasta pronto!', 'Cerrando sesión']
 > Processing time: 6.051150560379028
 > Real-time factor: 2.0429303934400656
 > Processing time: 6.051150560379028
 > Real-time factor: 2.0429303934400656


'audio_out/cerrando_programa.wav'